# Node Class

The Node class is a fundamental component of the logical expression parser and evaluator, designed to represent each element within a propositional logic formula as a structured entity within a parse tree

In [1]:
class Node:
    def __init__(self, value, children=None):
        self.value = value
        self.children = children if children else []
        self.id = None

    def evaluate(self, truth_assignment):
        if self.value.isalnum():  # Check if the node is a propositional variable
            return truth_assignment.get(self.value, False)
        elif self.value == '¬':  # Unary negation operator
            return not self.children[0].evaluate(truth_assignment)
        elif self.value == '∧':  # Logical AND operator
            return self.children[0].evaluate(truth_assignment) and self.children[1].evaluate(truth_assignment)
        elif self.value == '∨':  # Logical OR operator
            return self.children[0].evaluate(truth_assignment) or self.children[1].evaluate(truth_assignment)
        elif self.value == '→':  # Logical implication operator
            return not self.children[0].evaluate(truth_assignment) or self.children[1].evaluate(truth_assignment)

    def __str__(self):
        if self.children:
            return f"{self.value}({', '.join(map(str, self.children))})"
        return f"{self.value}"

# SAT Class

The SAT class is designed to handle the parsing, transformation, evaluation, and solving of propositional logic formulas represented as strings. This class is central to a software module that leverages tree data structures to model logical expressions and evaluate their satisfiability.

In [2]:
import itertools
import re
from random import randint
from math import log2


class SAT:

    def __init__(self, expression) -> None:
        self.expression = expression.replace('¬¬', '')
        self.atoms = re.findall(r'\w', expression)
        self.parse_tree = None
        self.transformed_tree = None
        self.last_id = 1
        self.ids = {}

    def parse(self):
        # Regular expression to match tokens: digits, words, parentheses, and operators
        token_pattern = r'\s*(?:(\d+|\w+|[()¬∧∨→]))\s*'
        tokens = re.findall(token_pattern, self.expression)
        # Stack for holding nodes that are being processed
        stack = []
        # Operator stack for handling operator precedence and parentheses
        ops = []

        def operator_precedence(op):
            """
                Returns the precedence of the logical operators, with higher values indicating higher precedence.

                Args:
                    op (str): A string representing one of the logical operators.

                Returns:
                    int: The precedence of the operator.
            """ 
            precedences = {'¬': 3, '∧': 2, '∨': 2, '→': 1}
            return precedences.get(op, -1)

        def process_operator(op):
            """
                Processes operators by popping the required number of operands from the stack, 
                applying the operator, and pushing the result back onto the stack.

                Args:
                    op (str): The operator to process.
            """
            if op == '¬':
                arg = stack.pop()
                stack.append(Node('¬', [arg]))
            elif op in {'∧', '∨', '→'}:
                right = stack.pop()
                left = stack.pop()
                stack.append(Node(op, [left, right]))

        # Process each token in the expression
        for token in tokens:
            if token.isalnum():  # Alphanumeric tokens are atoms
                stack.append(Node(token))
            elif token in {'¬', '∧', '∨', '→'}:
                # Handle operators with respect to precedence
                while (ops and ops[-1] != '(' and
                    operator_precedence(ops[-1]) >= operator_precedence(token)):
                    process_operator(ops.pop())
                ops.append(token)
            elif token == '(':
                # Opening parenthesis, push to stack to denote precedence change
                ops.append(token)
            elif token == ')':
                # Closing parenthesis, process all operators until the opening parenthesis
                while ops and ops[-1] != '(':
                    process_operator(ops.pop())
                ops.pop() # Remove the '(' from the stack

        # Process any remaining operators in the operator stack
        while ops:
            process_operator(ops.pop())
        
        self.parse_tree = stack[-1]
        return stack[-1]

    def _transform(self, node, ):
        # Handle atomic propositions directly without transformation
        if node.value in self.atoms:
            return Node(node.value, [])
        # Simplify negations by recursively transforming the child node
        elif node.value == '¬':
            subnode = self._transform(node.children[0])
            return Node('¬', [subnode])
        # Simplify conjunctions by recursively transforming each child
        elif node.value == '∧':
            left = self._transform(node.children[0])
            right = self._transform(node.children[1])
            return Node('∧', [left, right])
        # Convert disjunctions to negations of conjunctions (De Morgan's Law)
        elif node.value == '∨':
            left = self._transform(node.children[0])
            right = self._transform(node.children[1])
            return Node('¬', [Node('∧', [Node('¬', [left]), Node('¬', [right])])])
        # Convert implications to disjunctions using logical equivalences
        elif node.value == '→':
            left = self._transform(node.children[0])
            right = self._transform(node.children[1])
            return Node('¬', [Node('∧', [left, Node('¬', [right])])])
        else:
            return node  # Base case for other atomic propositions
        
    def transform(self):
        self.transformed_tree = self._transform(self.parse_tree)
        return self.transformed_tree

    def find_satisfying_assignments(self):
        """
            Computes all satisfying truth assignments for the propositional logic formula based on its parse tree.

            Returns:
                list: A list of dictionaries, each representing a truth assignment that satisfies the formula.

            Note:
                This function generates all possible combinations of truth values for the variables found in the formula
                and checks each combination to see if it satisfies the formula. The computational complexity of this approach
                is exponential with respect to the number of variables (2^n combinations).
        """
        vars_list = self.atoms
        satisfying_assignments = []
        for truth_assignment in itertools.product([False, True], repeat=len(vars_list)):
            assignment = dict(zip(vars_list, truth_assignment))
            if self.transformed_tree.evaluate(assignment):
                satisfying_assignments.append(assignment)
        
        self.satisfying_assignments = satisfying_assignments
        return satisfying_assignments
    
    def propagate_truth(self, node, assumed_true=True):
        """
            Recursively propagates truth values through the parse tree based on the assumption that
            the formula is true. This method assigns truth values to propositional variables and
            applies logical operations accordingly.

            Args:
                node (Node): The current node in the parse tree to process.
                assumed_true (bool): Indicates whether the current branch of the parse tree is 
                                    assumed to be true or false based on logical deductions.

            Note:
                This function operates in a linear time complexity with respect to the number of nodes,
                as it traverses each node exactly once. However, it might not find a solution if the 
                formula is unsatisfiable or if contradictions are encountered during propagation.
        """
        if self.falsity:
            return
        
        if node.value.isalnum() and self.ids.get(node.value):
            node.id = self.ids.get(node.value)
        elif node.value.isalnum():
            self.ids[node.value] = self.last_id
            node.id = self.last_id
            self.last_id += 1
        else:
            node.id = self.last_id
            self.last_id += 1

        if node.value == '∧' and assumed_true:
            # If AND and assumed true, both sides must be true
            for child in node.children:
                self.propagate_truth(child, True)
        elif node.value == '¬':
            # If NOT, propagate the opposite of the current assumption
            self.propagate_truth(node.children[0], not assumed_true)
        elif node.value in self.atoms:  # Base case for variables
            # Mark the variable based on the propagated truth value
            if self.assignments.get(node.value) and self.assignments.get(node.value) != assumed_true:
                self.falsity = True
                return
            self.assignments[node.value] = assumed_true

    def _generate_non_assigned_atoms(self, assignment):
        new_assignment = {}
        for atom in self.atoms:
            if assignment.get(atom) is None:
                new_assignment[atom] = True if randint(0, 1) == 1 else False
            else:
                new_assignment[atom] = assignment[atom]
        return new_assignment


    def solve(self):
        self.assignments = {}
        self.falsity = False
        self.propagate_truth(self.transformed_tree, True)
        if self.falsity:
            return False  # Return False if a contradiction is detected indicating unsatisfiability.
        if self.transformed_tree.evaluate(self.assignments) == True:
            return self.assignments  # Return assignments if the formula evaluates to True.
        
        # This section demonstrates the novelity of this work, taking the log2 of count of atoms, then add a constant like 5 to it and make guesses randomly 
        for _ in range(int(log2(len(self.atoms))) + 5):
            new_assignment = self._generate_non_assigned_atoms(self.assignments)
            if self.transformed_tree.evaluate(new_assignment) == True:
                return new_assignment  # Return assignments if the formula evaluates to True.
        return None  # Return None if the formula's truth cannot be determined.


    def _print_parse_tree(self, node, prefix="", is_tail=True):
        connector = "└── " if is_tail else "├── "
        if node.children:
            # Print the current node value
            print(prefix + connector + node.value)
            prefix += "    " if is_tail else "│   "
            # Recursively print each child, updating the prefix
            for i, child in enumerate(node.children):
                is_last_child = i == len(node.children) - 1
                self._print_parse_tree(child, prefix, is_last_child)
        else:
            # Print the leaf node value
            print(prefix + connector + node.value)
    
    def print_parse_tree(self):
        self._print_parse_tree(self.transformed_tree)

    def _print_dag(self, node, prefix="", is_tail=True, seen_ids=None):
        """
            Args:
                node (Node): The current node in the parse tree to be printed.
                prefix (str): A string prefix that represents the current level of indentation
                            for this node.
                is_tail (bool): A boolean that indicates if the node is the last child of its parent.
                seen_ids (set): A set that keeps track of node IDs that have already been printed
                                to manage the display of shared nodes.

            This method uses a recursive approach to traverse the tree. When it encounters a node,
            it first checks if the node's ID has been seen before:
                - If yes, it prints a reference to the shared node instead of recursing further.
                - If no, it prints the node details and proceeds to recursively print its children.

        """
        if seen_ids is None:
            seen_ids = set()
        connector = "└── " if is_tail else "├── "
        node_info = f"Node {node.id}: {node.value}"

        # Check if this node was printed before
        if node.id in seen_ids:
            print(f"{prefix}{connector}{node_info} (shared)")
        else:
            seen_ids.add(node.id)
            print(f"{prefix}{connector}{node_info}")
            children_prefix = "    " if is_tail else "│   "
            for i, child in enumerate(node.children):
                is_last_child = i == len(node.children) - 1
                self._print_dag(child, prefix + children_prefix, is_last_child, seen_ids)
    
    def print_dag(self):
        self._print_dag(self.transformed_tree)


# Test Class

The Test class serves as a utility for testing the satisfiability of propositional logic formulas using the SAT class. It encapsulates methods to evaluate a formula from parsing through to solving, providing a structured output that details each step of the process.

In [3]:
class Test:
    def __init__(self, original_formula) -> None:
        self.original_formula = original_formula

    def evaluate(self):
        sat_solver = SAT(self.original_formula)
        parsed_tree = sat_solver.parse()
        transformed_tree = sat_solver.transform()

        print("Original formula:", self.original_formula)
        print("Parsed tree:", parsed_tree)
        print("Translated formula:", transformed_tree)

        satisfying_assignments = sat_solver.solve()
        if satisfying_assignments:
            print("Satisfying assignments where the formula is true:")
            print(satisfying_assignments)
        elif satisfying_assignments == False:
            print("There are no satisfying assignments; the formula is always false.")
        else:
            print("The linear SAT solver could not find any model.")

        print("--- Parse Tree ---")
        sat_solver.print_parse_tree()

        print("-- DAG ---")
        sat_solver.print_dag()

# Main

In [4]:
from test import Test

def main():
    print("----- Test 1 -----")
    test1 = Test("p ∧ ¬(q ∨ ¬p)")
    test1.evaluate()

    print("----- Test 2 -----")
    test2 = Test("p ∧ ¬(q → ¬p)")
    test2.evaluate()

    print("----- Test 3 -----")
    test3 = Test("p ∧ ¬¬(¬q ∧ ¬¬p)")
    test3.evaluate()

    print("----- Test 4 -----")
    test4 = Test("p ∧ ¬p")
    test4.evaluate()

    print("----- Test 5 -----")
    test5 = Test("¬p ∧ (q ∨ r)")
    test5.evaluate()

    print("----- Test 6 -----")
    test6 = Test("¬p ∧ (q ∨ r) ∧ (s ∧ r)")
    test6.evaluate()


if __name__ == '__main__':
    main()

----- Test 1 -----
Original formula: p ∧ ¬(q ∨ ¬p)
Parsed tree: ∧(p, ¬(∨(q, ¬(p))))
Translated formula: ∧(p, ¬(¬(∧(¬(q), ¬(¬(p))))))
Satisfying assignments where the formula is true:
{'p': True, 'q': False}
--- Parse Tree ---
└── ∧
    ├── p
    └── ¬
        └── ¬
            └── ∧
                ├── ¬
                │   └── q
                └── ¬
                    └── ¬
                        └── p
-- DAG ---
└── Node 1: ∧
    ├── Node 2: p
    └── Node 3: ¬
        └── Node 4: ¬
            └── Node 5: ∧
                ├── Node 6: ¬
                │   └── Node 7: q
                └── Node 8: ¬
                    └── Node 9: ¬
                        └── Node 2: p (shared)
----- Test 2 -----
Original formula: p ∧ ¬(q → ¬p)
Parsed tree: ∧(p, ¬(→(q, ¬(p))))
Translated formula: ∧(p, ¬(¬(∧(q, ¬(¬(p))))))
Satisfying assignments where the formula is true:
{'p': True, 'q': True}
--- Parse Tree ---
└── ∧
    ├── p
    └── ¬
        └── ¬
            └── ∧
                ├── q
    